## Todo:
- Experiment with normalization while creating triplets
- Do you need dropout in the regressor network?
- Simultaneous feature learning?
- See when are results bad/equal/better? See which classes they correspond to.. 

In [ ]:
# General Imports
import tensorflow as tf
import numpy as np
import os, sys
from time import time
from matplotlib import pyplot as plt

from utils import optimistic_restore, save, load
import layers

In [ ]:
'''
HYPERPARAMS
'''
BATCH_SIZE = 10
DATA_PATH = '/media/red/capstone/data/cifar-100/feature_vectors.pkl'
INPUT_SIZE = '321,321'
LEARNING_RATE = 2.5e-4
BETA1 = 0.9
BETA2 = 0.99
NUM_CLASSES = 40
NUM_STEPS = 20001
RANDOM_SEED = 1234
RESTORE_FROM = './deeplab_resnet.ckpt'
SAVE_NUM_IMAGES = 2
SAVE_PRED_EVERY = 1000
SNAPSHOT_MAX = 10
SNAPSHOT_DIR = './snapshots/'
WEIGHT_DECAY = 0.0005

In [ ]:
'''
Load in Data
'''

# ANIMESSSSSHHHHHHHHHH   HAAAAALLLLPPPPPPP

In [ ]:
'''
Tensorflow Session Setup
'''
tf.set_random_seed(RANDOM_SEED)
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
init = tf.group(tf.global_variables_initializer(),
                tf.local_variables_initializer())
sess.run(init)

'''
Model Setup
'''

x = tf.placeholder(dtype=tf.float32, shape=(-1, 32, 32, 3))
y = tf.placeholder(dtype=tf.float32, shape=(-1, NUM_CLASSES))

net = []
net.append(layers.conv2d(
        input=x,
        shape=(3,3,3,32),
        padding='SAME',
        activation='relu',
        name='conv1_1'
        ))
net.append(layers.conv2d(
        input=net[-1],
        shape=(3,3,32,32),
        padding='SAME',
        activation='relu',
        name='conv1_2'
        ))
net.append(tf.nn.max_pool(
        value=net[-1],
        ksize=[1, 2, 2, 1],
        strides=[1,2,2,1],
        padding='SAME',
        name='pool_1'
        ))

net.append(layers.conv2d(
        input=net[-1],
        shape=(3,3,32,64),
        padding='SAME',
        activation='relu',
        name='conv2_1'
        ))
net.append(layers.conv2d(
        input=net[-1],
        shape=(3,3,64,64),
        padding='SAME',
        activation='relu',
        name='conv2_2'
        ))
net.append(tf.nn.max_pool(
        value=net[-1],
        ksize=[1, 2, 2, 1],
        strides=[1,2,2,1],
        padding='SAME',
        name='pool_2'
        ))

net.append(tf.contrib.layers.flatten(
        inputs=net[-1],
        scope='flat_1'
        ))
net.append(layers.fc(
        input=net[-1],
        units=1024,
        activation='relu',
        name='fc_1'
        ))
net.append(tf.nn.dropout(
        x=layers[-1],
        keep_prob=0.5,
        name='dropout_fc_1'
        ))
net.append(layers.fc(
        input=net[-1],
        units=1024,
        activation='relu',
        name='fc_2'
        ))
net.append(tf.nn.dropout(
        x=net[-1],
        keep_prob=0.5,
        name='dropout_fc_2'
        ))
net.append(layers.fc(
        input=net[-1],
        units=NUM_CLASSES,
        activation='linear',
        name='fc_3'
        ))
pred = tf.nn.softmax(
        logits=net[-1],
        name='softmax')

'''
Loss and Optimization Setup
'''
loss = tf.nn.softmax_cross_entropy_with_logits(
        labels=y, #GT probability distribution
        logits=net[-1], # unscaled log prob
        name='softmax_cross_entropy')

reduced_loss = tf.reduce_mean(loss)

optimizer = tf.train.AdamOptimizer(
        learning_rate=LEARNING_RATE,
        beta1=BETA1,
        beta2=BETA2,
        name='AdamOptimizer')
train_op = optimizer.minimize(reduced_loss)

'''
TensorBoard Setup
'''
all_summary = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(SNAPSHOT_DIR,
        graph=tf.get_default_graph())

'''
Tensorflow Saver Setup
'''
saver = tf.train.Saver(var_list=tf.global_variables(),
                       max_to_keep=SNAPSHOT_MAX)

'''
Main Training Loop
'''
for step in range(NUM_STEPS):
    start_t = time()
    # Fill this in!
    feed_dict = {x:,
                 y:}
    if step % SAVE_PRED_EVERY == 0:
        loss_v, _, summary_v  = sess.run([reduced_loss, train_op, all_summary],
                feed_dict=feed_dict)
        summary_writer.add_summary(reduced_loss, step)
        save(saver, sess, SNAPSHOT_DIR, step)
    else:
        loss_v, _ = sess.run([loss, train_op],
                feed_dict=feed_dict)
    duration = time() - start_t
    print('step {:d} \t loss = {:.3f}, ({:.3f} sec/step)'.format(
        step, loss_v, duration))
    